In [123]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import *
from IPython.display import display

In [124]:
movies=pd.read_csv(r'C:\Users\denit\OneDrive\Desktop\MYFOLDER\TM\Project\Datas\movies.csv')
ratings=pd.read_csv(r'C:\Users\denit\OneDrive\Desktop\MYFOLDER\TM\Project\Datas\ratings.csv')

In [125]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [126]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [127]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.2+ KB


In [128]:
movies.shape

(10329, 3)

In [129]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105339 entries, 0 to 105338
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  int64  
 1   movieId    105339 non-null  int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.2 MB


In [130]:
ratings.shape

(105339, 4)

In [131]:
ratings.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [132]:
movies['genres']=movies['genres'].str.split("|")

In [133]:
movies2=movies.explode('genres')
movies2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [134]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [135]:
movies2['genres'].nunique()

20

In [136]:
movies2=movies2[movies2['genres']!='(no genres listed)']

In [137]:
movies2['genres'].nunique()

19

In [138]:
merged_data=pd.merge(ratings,movies2,on=['movieId'],how='inner')
merged_data

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime
1,1,16,4.0,1217897793,Casino (1995),Drama
2,1,24,1.5,1217895807,Powder (1995),Drama
3,1,24,1.5,1217895807,Powder (1995),Sci-Fi
4,1,32,4.0,1217896246,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery
...,...,...,...,...,...,...
281892,668,143385,4.0,1446388585,Bridge of Spies (2015),Drama
281893,668,143385,4.0,1446388585,Bridge of Spies (2015),Thriller
281894,668,144976,2.5,1448656898,Bone Tomahawk (2015),Horror
281895,668,144976,2.5,1448656898,Bone Tomahawk (2015),Western


In [139]:
popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
popularity.columns=["Genres","Title","Average Ratings","NUmber of Ratings"]
popularity

,Genres,Title,Average Ratings,NUmber of Ratings
0,Action,'71 (2014),3.500000,1
1,Action,'Hellboy': The Seeds of Creation (2004),3.000000,1
2,Action,10 to Midnight (1983),2.500000,1
3,Action,12 Rounds (2009),2.875000,4
4,Action,13 Assassins (Jûsan-nin no shikaku) (2010),3.500000,5
...,...,...,...,...
23093,Western,Wyatt Earp (1994),3.200000,30
23094,Western,Young Guns (1988),3.375000,36
23095,Western,Young Guns II (1990),3.083333,12
23096,Western,Young Ones (2014),2.000000,1


In [140]:
popularity[(popularity['Genres']=='Action')&(popularity['NUmber of Ratings']==50)]

,Genres,Title,Average Ratings,NUmber of Ratings
1148,Action,Pirates of the Caribbean: Dead Man's Chest (2006),3.46,50


In [141]:
#genre=action
#threshold=50
#top=7
popularity[(popularity['Genres']=='Action')&(popularity['NUmber of Ratings']>=50)]

,Genres,Title,Average Ratings,NUmber of Ratings
13,Action,28 Days Later (2002),3.932836,67
20,Action,300 (2007),3.577922,77
43,Action,"Abyss, The (1989)",3.681319,91
62,Action,Air Force One (1997),3.439024,82
71,Action,Aliens (1986),4.146497,157
...,...,...,...,...
1638,Action,V for Vendetta (2006),4.015957,94
1676,Action,Waterworld (1995),2.934959,123
1692,Action,Wild Wild West (1999),2.164179,67
1707,Action,X-Men (2000),3.557692,130


In [142]:
popularity[(popularity['Genres']=='Action')&(popularity['NUmber of Ratings']>=50)].sort_values(by='Average Ratings')

,Genres,Title,Average Ratings,NUmber of Ratings
1692,Action,Wild Wild West (1999),2.164179,67
796,Action,Johnny Mnemonic (1995),2.453846,65
184,Action,Beverly Hills Cop III (1994),2.609091,55
800,Action,Judge Dredd (1995),2.640845,71
1024,Action,Mortal Kombat (1995),2.707547,53
...,...,...,...,...
1331,Action,Seven Samurai (Shichinin no samurai) (1954),4.217742,62
1433,Action,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
975,Action,"Matrix, The (1999)",4.264368,261
1076,Action,North by Northwest (1959),4.273973,73


In [143]:
popularity[(popularity['Genres']=='Action')&(popularity['NUmber of Ratings']>=50)].sort_values(by='Average Ratings',ascending=False).head(7)

,Genres,Title,Average Ratings,NUmber of Ratings
1179,Action,Princess Mononoke (Mononoke-hime) (1997),4.384615,52
1076,Action,North by Northwest (1959),4.273973,73
975,Action,"Matrix, The (1999)",4.264368,261
1433,Action,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
1331,Action,Seven Samurai (Shichinin no samurai) (1954),4.217742,62
1199,Action,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224
747,Action,Inception (2010),4.189320,103


In [144]:
#popularity recommender system


In [145]:
def TopNPopularMovies(genre,threshold,topN):
    popularity=merged_data.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
    popularity.columns=["Genres","Title","Average_Ratings","NUmber_of_Ratings"]

   #filter the data
    topNrecommendations=popularity[(popularity['Genres']==genre)&(popularity['NUmber_of_Ratings']>=threshold)].sort_values(by='Average_Ratings',ascending=False).head(topN)

   #output
    topNrecommendations['Sno.']=range(1,len(topNrecommendations)+1)
    topNrecommendations.index=range(0,len(topNrecommendations))
    topNrecommendations.columns=['Genres','Movie Title','Average Movie Rating','Number of Reviews','Sno.']
    return topNrecommendations[['Sno.','Movie Title','Average Movie Rating','Number of Reviews']]

In [146]:
#test case 1
genre='Adventure'
threshold=50
topN=8
TopNPopularMovies(genre=genre,threshold=threshold,topN=topN)

,Sno.,Movie Title,Average Movie Rating,Number of Reviews
0,1,Princess Mononoke (Mononoke-hime) (1997),4.384615,52
1,2,Monty Python and the Holy Grail (1975),4.301948,154
2,3,North by Northwest (1959),4.273973,73
3,4,Spirited Away (Sen to Chihiro no kamikakushi) ...,4.236111,72
4,5,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
5,6,Seven Samurai (Shichinin no samurai) (1954),4.217742,62
6,7,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224
7,8,Star Wars: Episode IV - A New Hope (1977),4.188645,273


In [147]:
#content based recommender system

In [ ]:
toy story>>>adventure,fantasy,children,comedy....

In [149]:
movies2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy


In [150]:
movies3=movies2.groupby('title').agg({"genres":lambda x:" ".join(list(x))}).reset_index()

In [151]:
movies3.head()

,title,genres
0,'71 (2014),Action Drama Thriller War
1,'Hellboy': The Seeds of Creation (2004),Action Adventure Comedy Documentary Fantasy
2,'Round Midnight (1986),Drama Musical
3,'Til There Was You (1997),Drama Romance
4,"'burbs, The (1989)",Comedy


In [152]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english')
tf

TfidfVectorizer(ngram_range=(1, 3), stop_words='english')

In [153]:
tf_matrix=tf.fit_transform(movies3['genres'])

In [154]:
cosine_sim=cosine_similarity(tf_matrix,tf_matrix)

In [155]:
cosine_sim

array([[1.        , 0.02677945, 0.02931913, ..., 0.10229517, 0.        ,
        0.        ],
       [0.02677945, 1.        , 0.        , ..., 0.03626651, 0.02411583,
        0.02863994],
       [0.02931913, 0.        , 1.        , ..., 0.        , 0.        ,
        0.35526663],
       ...,
       [0.10229517, 0.03626651, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.02411583, 0.        , ..., 0.        , 1.        ,
        0.07090711],
       [0.        , 0.02863994, 0.35526663, ..., 0.        , 0.07090711,
        1.        ]])

In [156]:
def recommendation_genre(movie_df,similarity_matrix,movie_title,topN):
    #indices for all movies
    indices=pd.Series(movies3.index,index=movies3['title'])
    #index of target movie
    index=int(indices[movie_title])
    #cosine similarity scores
    cosine_scores=list(enumerate (similarity_matrix[index]))
    cosine_scores=sorted(cosine_scores,key=lambda x:x[1],reverse=True)[:topN]
    #extracting matching movies
    matched=[i[0] for i in cosine_scores]
    matching_df=movie_df.iloc[matched]
    #filter out target movie
    matching_df=matching_df[matching_df['title']!=movie_title]


    #output
    matching_df.rename(columns={'title':'Movie Title'},inplace=True)
    matching_df['Sno.']=range(1,len(matching_df)+1)
    matching_df.index=range(0,len(matching_df))
    return matching_df[['Sno.','Movie Title']].head(topN)

In [157]:
#test case
recommendation_genre(movie_df=movies3,similarity_matrix=cosine_sim,movie_title='Toy Story (1995)',topN=10)

,Sno.,Movie Title
0,1,"Adventures of Rocky and Bullwinkle, The (2000)"
1,2,Antz (1998)
2,3,Asterix and the Vikings (Astérix et les Viking...
3,4,"Boxtrolls, The (2014)"
4,5,DuckTales: The Movie - Treasure of the Lost La...
5,6,"Emperor's New Groove, The (2000)"
6,7,"Monsters, Inc. (2001)"
7,8,Shrek the Third (2007)
8,9,"Tale of Despereaux, The (2008)"


In [158]:
#interactive widgets

In [159]:
set(movies2['genres'])

{'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [160]:
#popularity
#inputs
genres=Dropdown(options=list(set(movies2['genres'])),description="Genres",style={"description_width":"initial"})
num_reviews=IntText(description="Minimum Reviews",style={"description_width":'initial'})
num_recommendations_1=IntText(description="Number of Recommendations",style={"description_width":'initial'})

#tabs
b1=Button(description="RECOMMEND ME",style={"description_width":'initial'})
h1=HBox([num_reviews,num_recommendations_1])
popularity_tab=VBox([genres,h1,b1])

#content bases system
title=Textarea(description="Movie Title",style={"description_width":'initial'})
num_recommendations_2=IntText(description="Number of Recommendations",style={"description_width":'initial'})

#tabs
h2=HBox([title,num_recommendations_2])
b2=Button(description="RECOMMEND ME",style={"description_width":'initial'})
content_tab=VBox([h2,b2])

#creating final tabs
tabs=[popularity_tab,content_tab]
wid=widgets.Tab(tabs)

#set titles to the tabs
names=['popularity based recommendations','content based recommendations']
[wid.set_title(i,title) for i,title in enumerate(names)]
display(wid)

In [161]:
#setting up events to respond when clicked upon
output=Output()
def b1_clicked(b):
    global output
    with output:
        output.clear_output()
    output=TopNPopularMovies(genre=genres.value,threshold=num_reviews.value,topN=num_recommendations_1.value)
b1.on_click(b1_clicked)

#content
def b2_clicked(b):
    global output
    with output:
       output.clear_output() 
    result=recommendation_genre(movie_df=movies3,similarity_matrix=cosine_sim,movie_title=title.value,topN=num_recommendations_2.value)
    output=result
b1.on_click(b2_clicked)

In [162]:
display(wid)

In [165]:
output

,Sno.,Movie Title,Average Movie Rating,Number of Reviews
0,1,"Pier, The (Jetée, La) (1962)",4.625,4
1,2,Splendor in the Grass (1961),4.600,5
2,3,Blue Is the Warmest Color (La vie d'Adèle) (2013),4.500,4
